# 代码块

## 简介

**Block** 是 **Ruby** 的一个独特特性，它本质上就是一组代码，通过它，可以很容易实现回调，或传递一组代码（远比C的函数指针灵活），以及实现迭代器。这是一个不可思议的功能强大的特性。

语法表现上，**Block** 是花括号或者`do`和`end`之间的一组代码，按照社区约定，我们可以通过以下两种方式来书写 **Block**

- 多行，使用`do`和`end`作为开始和结束。
- 单行，使用花括号

In [5]:
[1, 2, 3].each do |i|
  puts i
end

[1, 2, 3].each{ |i| puts i }

1
2
3
1
2
3


[1, 2, 3]

## 概念

**Block** 相关的一些术语

- 闭包，`proc`，`lambda`, 函数，函数指针，匿名函数
- 回调，`callable`，`functor`，`delegate`

我们介绍下其中的几个概念。

#### 闭包

在计算机科学中，闭包（Closure）是词法闭包（Lexical Closure）的简称，是引用了自由变量的表达式（通常是函数）。这些被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。 词法作用域(lexical scope)等同于静态作用域(static scope)。所谓的词法作用域其实是指作用域在词法解析阶段既确定了，不会改变。

闭包的数据结构可以定义为，包含一个函数定义 f 和它定义时所在的环境 (struct Closure (f env))

- 全局函数是一个有名字但不会捕获任何值的闭包。
- 嵌套函数是一个有名字并可以捕获其封闭函数域内值得闭包。
- Lambda(闭包表达式)是一个利用轻量级语法所写的可以捕获其上下文中变量值的匿名闭包。

#### 函数指针

函数指针在`C`语言中用的比较多，本质上，它就是一个内存地址，只不过指向的是一块可执行代码的首地址。和闭包相比，它并没有附带定义该函数的上下文信息，也不负责指针类型检查。

#### functor

在`C++`中，`functor`是行为类似函数的对象，可以拥有成员函数和成员变量，即仿函数拥有状态，其本质和闭包很接近，只是用起来比较繁琐。

#### delegate

`C#`中的 `delegate` 类似于 `C` 或 `C++` 中的函数指针。使用 `delegate` 使程序员可以将方法引用封装在委托对象内。然后可以将该 ``delegate`` 对象传递给可调用所引用方法的代码，而不必在编译时知道将调用哪个方法。与C或C++中的函数指针不同，`delegate` 是面向对象、类型安全的，并且是安全的。

## 可调用对象(`callable object`)

**Ruby** 中，可调用对象是可执行的代码片段，它们都有自己的作用域，可调用对象有以下几种方式：

- **Block**，在定义它们的作用域中执行，它是闭包的一种。
- **proc**，`Proc` 类的对象，和块一样，它们也在定义自身的作用域中执行，它也是闭包的一种。
- **lambda**，也是 `Proc` 类的对象，和块一样，它们也在定义自身的作用域中执行，它和 `proc` 用法有细微的区别，也是闭包的一种。
- **Method**，绑定于对象，在所绑定对象的作用域中执行。

In [42]:
def capture_block(&block)
  block
end

level0_v = 'level0_v'
lambda0 = lambda do
  level1_v = 'level1_v'
  proc0 = proc do
    level2_v = 'level2_v'
    block0 = capture_block do
      level3_v = 'level3_v'
      puts format("level3: %s", binding.local_variables.grep(/level/))
    end
    puts format("level2: %s", binding.local_variables.grep(/level/))
    block0
  end
  puts format("level1: %s", binding.local_variables.grep(/level/))
  proc0
end

l2 = lambda0.call
l3 = l2.call
l3.call

lambda0.call.call.call

level1: [:level1_v, :level1, :level0_v]
level2: [:level2_v, :level1_v, :level1, :level0_v]
level3: [:level3_v, :level2_v, :level1_v, :level1, :level0_v]
level1: [:level1_v, :level1, :level0_v]
level2: [:level2_v, :level1_v, :level1, :level0_v]
level3: [:level3_v, :level2_v, :level1_v, :level1, :level0_v]


In [17]:
def capture_block(&block)
  block
end

def make_counter(n)
  capture_block do
    n -= 3
  end
end

c1 = make_counter(9)
c2 = make_counter(6)

puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"

c1 = 6, c2 = 3
c1 = 3, c2 = 0
c1 = 0, c2 = -3


In [18]:
def make_counter(n)
    lambda{ n -= 2 }
end

c1 = make_counter(9)
c2 = make_counter(6)

puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"

c1 = 7, c2 = 4
c1 = 5, c2 = 2
c1 = 3, c2 = 0


In [19]:
def make_counter(n)
    proc{ n -= 1 }
end

c1 = make_counter(9)
c2 = make_counter(6)

puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"
puts "c1 = #{c1.call}, c2 = #{c2.call}"

c1 = 8, c2 = 5
c1 = 7, c2 = 4
c1 = 6, c2 = 3


## yield

有别于其他语言，**Ruby** 中的 `yield` 是一种调用匿名函数的快捷方式。**Ruby** 有一种特殊的语法把匿名函数传递给一个方法，这种语法就是`Block`。

In [62]:
def twice
   yield
   yield
end

twice do
  puts "hi!"
end

hi!
hi!


**Ruby** 中对于所有方法，无论它的参数列表长什么样，它都可以在后面跟上一个可选的 `block` 参数。这个参数就叫做默认块，这个块可以使用 `yield` 来调用。

In [63]:
def add(a, b)
  a + b
end

add(1, 2) do
  puts "This code never called without yield!"
end

3

一个方法只能有一个 `block` 参数，如果显式地指定 `&block` 参数，则 `yield` 调用的就是 `&block` 传入的代码块。

In [69]:
def op(a, b, &block)
  puts block.call(a, b)
  puts yield a, b
end

op(1, 2) do|i, j|
  i + j
end

3
3


`&block` 只可以是最后一个方法参数，如果需要传入多个代码块到同一个方法，则其它对象需要使用 `Proc` 对象。

In [79]:
def block_first(&block, a)
end

SyntaxError: <main>: syntax error, unexpected ',', expecting ')'
def block_first(&block, a)
                       ^

In [81]:
def multi_block(&block1, &block2)
end

SyntaxError: <main>: syntax error, unexpected ',', expecting ')'
def multi_block(&block1, &block2)
                        ^

In [85]:
def ifthen(predict, *args)
  if predict.call(*args)
    yield
  else
    puts "Ignore"
  end
end

ifthen(lambda{ |m, n| m < n }, 1, 3) do
  puts "It is executed"
end

It is executed


## `proc` vs. `lamba`

`proc` 和 `lambda` 之间的差异可能是 **Ruby** 中最令人费解的特性。

In [48]:
def run(callable)
  puts callable.call
end

p = proc{ "Proc called" }
l = lambda{ "Lambda called" }

run(p)
run(l)

Proc called
Lambda called


在 **Ruby** 中，最后一行的 `return` 关键字是经常可以省略的，但是在 `proc` 和 `lambda` 当中，加上 `return` 关键字后，他们的行为是不一致的。

在 `lambda` 中，`return` 仅仅表示从这个 `lambda` 中返回，在 `proc` 当中，它不是从 `proc` 中返回，而是从定义 `proc` 的上下文中返回。

In [52]:
def run(callable)
  puts callable.call
end

p = proc{ return "Proc called" }
l = lambda{ return "Lambda called" }

begin 
  run(p)  
rescue LocalJumpError => e
  puts "proc executed error #{e.message}."
end

run(l)

proc executed error unexpected return.
Lambda called


`proc` 的行为比较诡异，更好的设计应该是从 `run` 方法返回，`lambda` 可以认为是一个方法调用，而 `proc` 则相当于把代码块插入到调用的位置去执行。

```ruby
def run(callable)
    callable.call
    ...
end

p = proc{ return "Proc called" }

run(p)
```

如果不考虑闭包的绑定，展开的过程应该这样的。

```ruby
def run(callable)
    return "Proc called"
    ...
end

run
```

当执行到 `return "Proc called"` 的时候，原本应该返回到 `run` 的返回地址，但是因为 `proc` 记录的确是定义它的 `main` 返回地址，因此报错，它等价于如下的代码。

In [55]:
begin 
  return "Proc called"
rescue LocalJumpError => e
  puts "proc executed error #{e.message}."
end

proc executed error unexpected return.


不过只要我们弄明白了这个坑点，倒是可以用来做一些有意思的事情。

In [61]:
def count_down_to(n)
  sentry = proc do|n|
    if n <= 0
      return "count down finished by sentry!"
    end
  end
  
  10.downto(n) do|i|
    print i, ', '
    sentry.call(i)
  end
  
  return "count down finish!"
end

puts count_down_to(3)
puts count_down_to(-3)

10, 9, 8, 7, 6, 5, 4, 3, count down finish!
10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, count down finished by sentry!


我们再来看 `block`，它的行为更像 `proc`，所以一般我们最好不要在 `block` 块中使用 `return`，除非你明确知道自己在干什么。

In [53]:
def capture_block(&block)
  block
end

b1 = capture_block{ "Block called" }
b2 = capture_block{ return "Block called" }

begin 
  run(b1)
  run(b2)
rescue LocalJumpError => e
  puts "block executed error #{e.message}."
end

Block called
block executed error unexpected return.


`proc` 和 `lambda` 还有点重要的区别来自他们检查参数的方式，`lambda` 总是检查传入的参数数量，如果和定义的不匹配，会抛出一个 `ArgumentError`。而 `proc` 则会把传递进来的参数调整为自己期望的参数形式，如果参数比期望的要多，`proc` 会忽略多余的参数，如果参数数量不足，那么对未指定的参数，proc会赋一个 `nil` 值。

整体而言，`lambda` 更直观，也更像一个方法，它不仅对参数数量要求严格，而且在调用 `return` 时，只在 `lambda` 的代码块返回。基于这些原因，如没有使用到某些 `proc` 的特殊功能，应该总是优先选择使用 `lambda`。